Grading for pre-lecture questions is all or nothing. Partial credit is available for in-class assignments and checkpoints. Partial credit will primarily be determined by the quality of the comments you make in your code. Tell me what you intend the code to do. If there are no comments, there is no partial credit!

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Learning Objectives

This lecture will show you how to:
1. Use `scipy.fft`
2. Interpret the relationship between aliasing and the Nyquist frequency
3. Know when to use the discrete Cosine transform
4. Apply Fourier smoothing to noisy data

In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt

from scipy import fft # Fourier transforms

import grading_helper as _test

# Using `scipy.fft.fft`

Update: As of SciPy 1.4, `scipy.fft` is favored in place of `scipy.fftpack`. The code in the video will all behave exactly the same if you change `fftpack` to `fft`.

In [ ]:
%video 6U226pAQixY

Summary:

- `scipy.fft.fft` is the Fast Fourier Transform (FFT).
- In brief, this algorithm exploits the fact that the DFT of a single point is the point itself. Additionally, the inverse DFTs of A and B are related to the inverse DFT of AB using simple operations (discussed more in your textbook). The end effect is that the FFT can transform the entire array in place, rather than needing to calculate one term at a time.
- The algorithm is fastest when $N$ is a power of 2, and slowest when $N$ is prime (although the difference only matters if the array is huge).
- The inverse transform is `scipy.fft.ifft`

## Your Turn

Use `fft` to calculate the discrete Fourier transform of the array `y` below. Store the result in an array named `c`.

In [ ]:
y = np.array([8, 6, 7, 5, 3, 0, 9])

In [ ]:
%%graded # 1 point

# YOUR CODE HERE

In [ ]:
%%tests

_test.similar(c, [38.0+0.00j, 8.59-5.35j, 3.34+7.53j, -2.93+4.82j, -2.93-4.82j, 3.34-7.53j, 8.59+5.35j])

# Aliasing and the Nyquist Frequency

In [ ]:
%video VI98XaHhkQM

Summary:

- When the array $y$ is **real**, the upper half of the coefficients of the Fourier transform are complex conjugates of the lower half. The very first coefficient is an exception: it's the sum of the array (or $N\,\times$ the average value). That is, only the first `N//2 + 1` coefficients are unique. When the input is real, it's conventional to only plot frequencies up to the symmetry point.
- The symmetry point is called the **Nyquist frequency** $f_{Ny}$. It represents the highest frequency that can be resolved. (Be careful about the difference between frequency $f$ and angular frequency $\omega=2\pi f$.)
- For real $y$, frequencies that are $\Delta f$ above the Nyquist frequency are indistinguishable from an equivalent lower frequency. This effect is called **aliasing**. (We didn't see this issue with the continuous Fourier transform, because the Nyquist frequency is infinite in that case.)
- Let $\Delta x$ be the spacing between the $N$ samples. The Nyquist frequency is
$$f_{Ny} = \frac{1}{2\Delta x}\,.$$
The frequency resolution is
$$\Delta f = \frac{1}{N \Delta x}\,.$$

## Your Turn

The array below represents data collected every 0.01 seconds. Use `fft` to plot the frequency-content of the data up to, and including, the Nyquist frequency. In principle, there should be two peaks: 30 Hz, and 60 Hz. Instead, you'll see one peak at 30 Hz, and another at 40 Hz. That happens because 60 Hz is above the Nyquist frequency.

Store the value of the Nyquist frequency in a variable named `fNy`.

In [ ]:
t = np.arange(0, 10, 0.01)
data = 3*np.sin(60*np.pi*t) - 4*np.cos(120*np.pi*t)

In [ ]:
%%graded # 2 points

# YOUR CODE HERE

In [ ]:
%%tests

_test.similar(fNy, 50)
_test.plot_shown()

# Discrete Cosine Transform

Update: It's no longer necessary to use `norm="otho"` as long as you use `scipy.fft` instead of `scipy.fftpack`. The `norm` determines whether the factor of $1/N$ goes with `dct`, `idct`, or is split $1/\sqrt{N}$ for each (which is what `norm="otho"` does). You may still encounter this strangeness in older code, so it's good to be aware that there are different normalization conventions.

In [ ]:
%video n0AZwyDnJeM

Summary:

- A Fourier transform implicitly assumes that the function being transformed is periodic. If the first and last point have similar values, it will work well. If they are instead significantly different, it will work poorly near the ends.
- The Cosine transform mirrors the function at its end points, which forces it to be periodic. The price paid is that this form assumes that the slope at the end points is zero.
- The DCT only works for real functions, and the result is always real.
- `scipy.fft.dct` is the DCT and `scipy.fft.idct` is the inverse.

# Fourier Filtering and Smoothing

In [ ]:
%video kCSSHxOCKyA

Summary:

- Since a DFT (or DCT) represents data in terms of its frequencies, we can eliminate some sources of noise of stripping the high frequency terms.
- Similarly, we can smooth functions by only keeping the terms with the largest magnitude.
- Recipe:
    1. Transform data
    2. Set unwanted frequencies to zero
    3. Inverse transform data.

## Your Turn

Calculate the Cosine transform of the array `y` below. Identify the 8 Fourier coefficients with the **largest magnitude**. Calculate the inverse transform using just these largest coefficients, with the rest set to zero. Plot the resulting array on top of the original.

> Hint: The built-in `sorted` function will come in handy.
> Hint: If using `scipy.fftpack` remeber to use `norm="otho"`. Otherwise, it doesn't matter, since you will undo the transformation

In [ ]:
y = np.array([0.00000000e+00, 1.66223040e-02, 1.20200612e-01, 3.41736915e-01, 6.32643608e-01, 8.87294265e-01,
              9.99246960e-01, 9.19355200e-01, 6.82811325e-01, 3.89771185e-01, 1.50559197e-01, 2.65364391e-02,
              8.99362093e-05, -9.47880272e-03, -9.36102819e-02, -2.95945994e-01, -5.81385008e-01, -8.50778601e-01,
              -9.93238506e-01, -9.46520284e-01, -7.31248690e-01, -4.39558718e-01, -1.84600924e-01, -3.95603883e-02,
              -7.17324361e-04, 4.71057490e-03, 7.07933740e-02, 2.52831117e-01, 5.29673941e-01, 8.10306521e-01,
              9.81305898e-01, 9.68413275e-01])

In [ ]:
%%graded # 2 points

# YOUR CODE HERE

In [ ]:
%%tests

_test.code_contains("dct", "idct")
_test.plot_shown()

# Additional Resources

- Textbook sections 7.3 and 7.4

Section 7.4 runs through a derivation of the Fast Fourier Transform.